In [1]:
from PyQt5 import QtGui  # (the example applies equally well to PySide)
import pyqtgraph as pg
import pandas as pd
import numpy as np

C:\Users\nshawen\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
## Always start by initializing Qt (only once per application)
app = QtGui.QApplication([])

In [153]:
class AccViewer(QtGui.QDialog):
    def __init__(self, parent=None):
        super(AccViewer,self).__init__(parent)
        
        self.WindowRange = 8 # time (s) to add to both sides of acc signal
        
        ## Load timestamp data
        Labels = pd.read_hdf('Y:\CIS-PD MUSC\decoded_forms\\form509.h5')
        Questions = pd.read_sas('Y:\CIS-PD MUSC\datadictionary.sas7bdat')
        Questions = Questions[Questions.zFormID == 509.0]
        Questions = Questions.drop(columns = list(set(Questions.columns)-set(['zFieldNm','zFieldNb'])))
        Questions['zFieldNm'] = Questions['zFieldNm'].astype(str)
        Questions.zFieldNm = Questions.zFieldNm.str.replace(('b\''), '')
        Questions.zFieldNm = Questions.zFieldNm.str.replace(('\''), '')
        Questions['zFieldNb'] = Questions['zFieldNb'].astype(str)
        Questions.zFieldNb = Questions.zFieldNb.str.replace(('b\''), '')
        Questions.zFieldNb = Questions.zFieldNb.str.replace(('\''), '')        
        
        self.questions = Questions
        self.timestamps = Labels
        
        ## Create some widgets to be placed inside
        self.btn = QtGui.QPushButton('Plot Data')
        self.subjtxt = QtGui.QLineEdit('1004')
        self.sesstxt = QtGui.QLineEdit('2 Weeks: Time 30')
        self.tasktxt = QtGui.QLineEdit('Walking')
        self.listw = QtGui.QListWidget()
        self.plot = pg.PlotWidget()

        self.btn.clicked.connect(self.btn_clicked)
        

        ## Create a grid layout to manage the widgets size and position
        layout = QtGui.QGridLayout()

        ## Add widgets to the layout in their proper positions
        layout.addWidget(self.btn, 0, 0)   # button goes in upper-left
        layout.addWidget(self.subjtxt, 1, 0)   # text edit goes in middle-left
        layout.addWidget(self.sesstxt, 2, 0)
        layout.addWidget(self.tasktxt, 3, 0)
        layout.addWidget(self.listw, 4, 0)  # list widget goes in bottom-left
        layout.addWidget(self.plot, 0, 1, 5, 1)  # plot goes on right side, spanning 3 rows

        self.setLayout(layout)

    def btn_clicked(self):
        sessInds = (self.timestamps.SubjectCode.astype(int) == int(self.subjtxt.text())
                   ) & (self.timestamps.VisitNm == self.sesstxt.text())
        
        watchTime = self.timestamps[sessInds]['Q146_UTC'].values[0]
        print(watchTime)
        
        StQinds = self.questions.zFieldNm == (self.tasktxt.text() + ' start time')
        StQNb = self.questions[StQinds].zFieldNb
        stTime = self.timestamps[sessInds][StQNb].values[0]
        stTime = watchTime + self.getTimeDelta(stTime[0])
        
        SpQinds = self.questions.zFieldNm == (self.tasktxt.text() + ' stop time')
        SpQNb = self.questions[SpQinds].zFieldNb
        spTime = self.timestamps[sessInds][SpQNb].values[0]
        spTime = watchTime + self.getTimeDelta(spTime[0])
        
        ACC = pd.read_csv(self.subjtxt.text() + '_Session1_Acc.csv', parse_dates=[3])
        ACC = ACC[(ACC.timestamp < (spTime + np.timedelta64(self.WindowRange,'s'))
                  ) & (ACC.timestamp > (stTime - np.timedelta64(self.WindowRange,'s')))]

        
        T = (ACC.timestamp.values.astype('datetime64[ms]') - ACC.timestamp.values[0].astype('datetime64[ms]')).astype(float)
        
        self.plot.plot(T,ACC.x.values,pen=(1,5))
        self.plot.plot(T,ACC.y.values,pen=(2,5))
        self.plot.plot(T,ACC.z.values,pen=(3,5))
        
        stTime = stTime.astype('datetime64[ms]').astype(float) - ACC.timestamp.values[0].astype('datetime64[ms]').astype(float)
        spTime = spTime.astype('datetime64[ms]').astype(float) - ACC.timestamp.values[0].astype('datetime64[ms]').astype(float)
        
        self.plot.plot(np.array([stTime,stTime]),np.array([-3,3]),pen=(4,5))
        self.plot.plot(np.array([spTime,spTime]),np.array([-3,3]),pen=(5,5))
        self.plot.setYRange(-3,3)
        
        
    def getTimeDelta(self, ts): #Convert string stopwatch time to timedelta
         return np.timedelta64(int(ts[0:2]),'h') + np.timedelta64(int(ts[3:5]),'m') + np.timedelta64(int(ts[6:]),'s')
        
    

In [140]:
s = np.datetime64('2017-07-07T14:25:29.000000000')

In [141]:
np.array([s,s])

array(['2017-07-07T14:25:29.000000000', '2017-07-07T14:25:29.000000000'],
      dtype='datetime64[ns]')

In [ ]:
window = AccViewer()
window.show()
app.exec_()

2017-07-07T14:22:59.000000000


In [105]:
np.datetime64('2017-07-07T14:25:54.000000000')

numpy.datetime64('2017-07-07T14:25:54.000000000')

In [107]:
ACC.timestamp < np.datetime64('2017-07-07T14:25:54.000000000') ACC.timestamp < np.datetime64('2017-07-07T14:25:54.000000000')

0           True
1           True
2           True
3           True
4           True
5           True
6           True
7           True
8           True
9           True
10          True
11          True
12          True
13          True
14          True
15          True
16          True
17          True
18          True
19          True
20          True
21          True
22          True
23          True
24          True
25          True
26          True
27          True
28          True
29          True
           ...  
2677005    False
2677006    False
2677007    False
2677008    False
2677009    False
2677010    False
2677011    False
2677012    False
2677013    False
2677014    False
2677015    False
2677016    False
2677017    False
2677018    False
2677019    False
2677020    False
2677021    False
2677022    False
2677023    False
2677024    False
2677025    False
2677026    False
2677027    False
2677028    False
2677029    False
2677030    False
2677031    False
2677032    Fal

In [79]:
l = Labels['Q16'][0]

In [80]:
l

'00:02:55'

In [81]:
np.timedelta64(int(l[0:2]),'m') + np.timedelta64(int(l[3:5]),'s') + np.timedelta64(int(l[6:]),'ms')

numpy.timedelta64(2055,'ms')

In [49]:
Questions = pd.read_sas('Y:\CIS-PD MUSC\datadictionary.sas7bdat')
Questions = Questions[Questions.zFormID == 509.0]
Questions = Questions.drop(columns = list(set(Questions.columns)-set(['zFieldNm','zFieldNb'])))

In [50]:
Questions['zFieldNm'] = Questions['zFieldNm'].astype(str)
Questions['zFieldNb'] = Questions['zFieldNb'].astype(str)

In [54]:
Questions[Questions.zFieldNm == 'Walking start time']

,zFieldNb,zFieldNm
317,b'Q16',Walking start time


In [53]:
Questions.zFieldNm = Questions.zFieldNm.str.replace(('b\''), '')
Questions.zFieldNm = Questions.zFieldNm.str.replace(('\''), '')

In [132]:
Labels.SubjectCode.unique()

array([1003., 1004., 1005., 1007., 1009., 1016., 1018., 1019., 1020.,
       1023., 1024., 1029., 1030., 1032., 1038., 1039., 1043., 1044.,
       1046., 1047., 1048., 1049., 1050., 1051., 1052., 1053., 1054.,
       1055., 1056.])

In [67]:
Questions.zFieldNb[305]

"b'Q146'"

In [10]:
Labels = pd.read_hdf('Y:\CIS-PD MUSC\decoded_forms\\form509.h5')
Questions = pd.read_sas('Y:\CIS-PD MUSC\datadictionary.sas7bdat')

In [120]:
Questions.zFieldNm

300                                       Data Collected
301                                   Date of assessment
302                   What is the participant\x92s state
303    On which hand is the subject wearing the Apple...
304    Was the device shaken 5 times up and down prio...
305    Patient\x92s Apple watch clock time when the s...
306                                   Shaking start time
307                                     Shaking end time
308                                Was standing assessed
309                                  Standing start time
310                                    Standing end time
311                               Standing overall score
312           Standing dyskinesia left upper limb rating
313          Standing dyskinesia right upper limb rating
314               Standing tremor left upper limb rating
315              Standing tremor right upper limb rating
316                                 Was walking assessed
317                            

In [20]:
Questions['zFieldNm'] = Questions['zFieldNm'].astype(str)

In [21]:
type(Questions[Questions.zFormID == 509.0].zFieldNm.values[0])

str

In [14]:
Questions[Questions.zFieldNm == 'b\'Walking start time\'']

,zFormID,zCRFNb,zFieldNb,zFieldNm,zSASNm,zSASLabel,zFieldTypeNm,zLength,zUnit,zRequired,zCodeGroup


In [8]:
T = ACC.timestamp.values.astype('datetime64[ms]') - ACC.timestamp.values[0].astype('datetime64[ms]')
T.astype(float)

array([0.000000e+00, 2.000000e+01, 4.000000e+01, ..., 8.639993e+07,
       8.639995e+07, 8.639997e+07])

In [34]:
np.median(np.diff(T))

numpy.timedelta64(20000000,'ns')

In [6]:
AccViewer()

In [104]:
ACC = pd.read_csv('1004_Session1_Acc.csv', parse_dates=[3])

In [33]:
ACC.head(10)

,user_id,experiment,cohort,timestamp,x,y,z
0,142579,Clinicians,northwestern,2017-07-07 00:00:00.014,0.695801,0.698975,-0.160889
1,142579,Clinicians,northwestern,2017-07-07 00:00:00.034,0.696777,0.703857,-0.161377
2,142579,Clinicians,northwestern,2017-07-07 00:00:00.054,0.696289,0.701416,-0.159912
3,142579,Clinicians,northwestern,2017-07-07 00:00:00.074,0.696289,0.699951,-0.162354
4,142579,Clinicians,northwestern,2017-07-07 00:00:00.094,0.696777,0.708740,-0.160889
5,142579,Clinicians,northwestern,2017-07-07 00:00:00.114,0.700195,0.703857,-0.162842
6,142579,Clinicians,northwestern,2017-07-07 00:00:00.134,0.701660,0.697510,-0.160889
7,142579,Clinicians,northwestern,2017-07-07 00:00:00.154,0.702148,0.708740,-0.157959
8,142579,Clinicians,northwestern,2017-07-07 00:00:00.174,0.704590,0.705811,-0.158936
9,142579,Clinicians,northwestern,2017-07-07 00:00:00.194,0.701172,0.699463,-0.162842
